In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
%matplotlib inline
import pickle
import gensim
import seaborn as sns
import sqlite3
import nltk
from nltk.corpus import stopwords
import re 
from sklearn.feature_extraction.text import CountVectorizer  ## BOW Model
from sklearn.feature_extraction.text import TfidfVectorizer  ## TFIDF Model

from sklearn.manifold import TSNE    ## To visualize high dimensional data


C:\Users\shravan\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
con=sqlite3.connect("data.sqlite")
data=pd.read_sql_query("""
select * from Spam 

""",con)
con.close()

In [5]:
data

,text,class
0,"> deat : sun , 15 dec 91 2 : 25 : 2 est > : mi...",0.0
1,"discussion s - > np + np remind ago read , sou...",0.0
2,". . . 's much restrictive s - > np np . 's "" ""...",0.0
3,""" listserv "" international conference 1992 sec...",0.0
4,anyone point book article causative constructi...,0.0
5,graduate student education approach colleague ...,0.0
6,undergraduate course shortly teach psycholingu...,0.0
7,on-line corpus modern german . information app...,0.0
8,"hi , help ! design experiment mandarin tone pa...",0.0
9,linguistic program university utah invite appl...,0.0


In [6]:
stop = set(stopwords.words('english')) #set of stopwords

In [7]:
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

In [8]:
def cleanhtml(sentence):
    '''This function removes all the html tags in the given sentence'''
    cleantext = re.sub('<.*?>', ' ', sentence)  ## Substitute <space> in place of any html tag
    return cleantext

In [9]:
def cleanpunc(sentence):
    '''This function cleans all the punctuation or special characters from a given sentence'''
    cleaned = re.sub(r'[?|@|!|^|%|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned

In [12]:
def preprocessing(series):
    '''The function takes a Pandas Series object containing text in all the cells
       And performs following Preprocessing steps on each cell:
       1. Clean text from html tags
       2. Clean text from punctuations and special characters
       3. Retain only non-numeric Latin characters with lenght > 2
       4. Remove stopwords from the sentence
       5. Apply stemming to all the words in the sentence
       
       Return values:
       1. final_string - List of cleaned sentences
       2. list_of_sent - List of lists which can be used as input to the W2V model'''
    
    i = 0
    str1=" "
    final_string = []    ## This list will contain cleaned sentences
    list_of_sent = []    ## This is a list of lists used as input to the W2V model at a later stage
    
    
    
    for sent in series.values:
        ## 
        filtered_sent = []
        sent = cleanhtml(sent)    ## Clean the HTML tags
        sent = cleanpunc(sent)    ## Clean the punctuations and special characters
        ## Sentences are cleaned and words are handled individually
        for cleaned_words in sent.split():
            ## Only consider non-numeric words with length at least 3
            if((cleaned_words.isalpha()) and (len(cleaned_words) > 2)):
                ## Only consider words which are not stopwords and convert them to lowet case
                if(cleaned_words.lower() not in stop):
                    ## Apply snowball stemmer and add them to the filtered_sent list
                    s = (sno.stem(cleaned_words.lower()))#.encode('utf-8')
                    filtered_sent.append(s)    ## This contains all the cleaned words for a sentence
                  ##  if (final['Score'].values)[i] == 'positive':
                    ##    all_positive_words.append(s) #list of all words used to describe positive reviews
                    ##if(final['Score'].values)[i] == 'negative':
                      ##  all_negative_words.append(s) #list of all words used to describe negative reviews
        ## Below list is a list of lists used as input to W2V model later
        list_of_sent.append(filtered_sent)
        ## Join back all the words belonging to the same sentence
        str1 = " ".join(filtered_sent)
        ## Finally add the cleaned sentence in the below list
        final_string.append(str1)
        #print(i)
        i += 1
    return final_string, list_of_sent

In [13]:
## This takes around 1 hour
final_string, list_of_sent=preprocessing(data["text"])

In [15]:
len(final_string)

962

In [16]:
data["cleaned_text"]=final_string

In [17]:
data

,text,class,cleaned_text
0,"> deat : sun , 15 dec 91 2 : 25 : 2 est > : mi...",0.0,deat sun dec est michael subject queri wlodek ...
1,"discussion s - > np + np remind ago read , sou...",0.0,discuss remind ago read sourc forget critiqu n...
2,". . . 's much restrictive s - > np np . 's "" ""...",0.0,much restrict pro quit
3,""" listserv "" international conference 1992 sec...",0.0,listserv intern confer second circular februar...
4,anyone point book article causative constructi...,0.0,anyon point book articl causat construct korea...
5,graduate student education approach colleague ...,0.0,graduat student educ approach colleagu mine qu...
6,undergraduate course shortly teach psycholingu...,0.0,undergradu cours short teach psycholinguist ap...
7,on-line corpus modern german . information app...,0.0,corpus modern german inform appreci ken beesle...
8,"hi , help ! design experiment mandarin tone pa...",0.0,help design experi mandarin tone part phonolog...
9,linguistic program university utah invite appl...,0.0,linguist program univers utah invit applic vis...


In [18]:
with open("list_of_sent","wb") as pkl:
    pickle.dump(list_of_sent,pkl)

In [19]:
conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
data.to_sql('Spam', conn, if_exists='replace', index = False)
conn.close()

In [20]:
data

,text,class,cleaned_text
0,"> deat : sun , 15 dec 91 2 : 25 : 2 est > : mi...",0.0,deat sun dec est michael subject queri wlodek ...
1,"discussion s - > np + np remind ago read , sou...",0.0,discuss remind ago read sourc forget critiqu n...
2,". . . 's much restrictive s - > np np . 's "" ""...",0.0,much restrict pro quit
3,""" listserv "" international conference 1992 sec...",0.0,listserv intern confer second circular februar...
4,anyone point book article causative constructi...,0.0,anyon point book articl causat construct korea...
5,graduate student education approach colleague ...,0.0,graduat student educ approach colleagu mine qu...
6,undergraduate course shortly teach psycholingu...,0.0,undergradu cours short teach psycholinguist ap...
7,on-line corpus modern german . information app...,0.0,corpus modern german inform appreci ken beesle...
8,"hi , help ! design experiment mandarin tone pa...",0.0,help design experi mandarin tone part phonolog...
9,linguistic program university utah invite appl...,0.0,linguist program univers utah invit applic vis...
